In [ ]:
import numpy as np
import tensorflow as tf
import os

from triplet_dataset_loader import *
from TL_class import SiameseTripletModel
from TL_siamese_network import generate_siamese_triplet_network
from top_accuracy_functions import *

In [ ]:
database_path = "../databases/my-fruit-recognition"
split_ratio = (0.8, 0.1, 0.1)
image_size = (100,100)
batch_size = 32

auto = tf.data.AUTOTUNE

learning_rate = 0.0001
steps_per_epoch = 50
validation_steps = 10
epochs = 5

# Model Creation & Training

In [ ]:
train_dataset, val_dataset, test_dataset = create_dataset(database_path, split_ratio, image_size, batch_size)

triplet_siamese_network = generate_siamese_triplet_network(image_size)

triplet_siamese_model = SiameseTripletModel(triplet_siamese_network)

triplet_siamese_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate))
triplet_siamese_model.summary()

In [ ]:
triplet_siamese_model.load_weights("5epoch_model.weights.h5")

In [ ]:
# history = triplet_siamese_model.fit(
#     train_dataset,
#     validation_data=val_dataset,
#     steps_per_epoch=steps_per_epoch,
#     validation_steps=validation_steps,
#     epochs=epochs
# )

In [ ]:
# triplet_siamese_model.save_weights('5epoch_model.weights.h5')

# Extracting embedding layers

In [ ]:
embedding_model = triplet_siamese_network.get_layer("Embedding")

## Getting reference images and their embeddings

In [ ]:
# Define the base path for the test images
base_test_folder = "../databases/my-fruit-recognition"

# Get the first image path from each subfolder and their class names
first_image_paths = []
class_names = []

for root, dirs, files in os.walk(base_test_folder):
    files = [f for f in files if f != ".DS_Store"]
    if files:
        files_sorted = sorted(files)
        first_image_path = os.path.join(root, files_sorted[0])
        first_image_paths.append(first_image_path)
        class_names.append(os.path.basename(root))

# Initialize the MapFunction to preprocess the images
mapF = MapFunction(image_size)
# Preprocess and convert the first images to tensors
first_images = [mapF.decode_and_resize(image_path) for image_path in first_image_paths]
first_images = tf.convert_to_tensor(first_images)
# Generate embeddings for the first images
first_embeddings = embedding_model.predict(first_images)

# Print the class names and shape of the embeddings
print("Class Names:", class_names)
print("Embeddings Shape:", first_embeddings.shape)

## Get query images and predict their embeddings

In [ ]:
query_image_path = "../databases/my-fruit-recognition/Banana/Banana01.png"
query_image = mapF.decode_and_resize(query_image_path)

query_image = tf.expand_dims(query_image, axis=0)  # Add batch dimension
query_embedding = embedding_model.predict(query_image)

## Top 1 accuracy

In [ ]:
closest_vector = top1_accuracy(query_embedding, first_embeddings)
closest_image_index = np.argmin([euclidean_distance(closest_vector, vec) for vec in first_embeddings])

In [ ]:
show_top1_accuracy(first_image_paths, closest_image_index, query_image_path, mapF)

## Top 3 accuracy

In [ ]:
closest_3_vectors = top3_accuracy(query_embedding, first_embeddings)

closest_3_indices = [np.argmin([euclidean_distance(vec, ref_vec) for ref_vec in first_embeddings]) for vec in closest_3_vectors]

In [ ]:
show_top3_accuracy(first_image_paths, closest_3_indices, query_image_path, mapF)

## Embeddings for test dataset

In [ ]:
small_test_folder = "../databases/my-fruit-recognition-small"

# Get all image paths in the smaller test dataset
small_test_image_paths = []
for root, dirs, files in os.walk(base_test_folder):
    files = [f for f in files if f != ".DS_Store"]
    if files:
        for file in files:
            small_test_image_paths.append(os.path.join(root, file))

# Initialize the MapFunction to preprocess the images
mapF = MapFunction(image_size)
# Preprocess and convert the test images to tensors
small_test_images = [mapF.decode_and_resize(image_path) for image_path in small_test_image_paths]
small_test_images = tf.convert_to_tensor(small_test_images)
# Generate embeddings for the test images
small_test_embeddings = embedding_model.predict(small_test_images)

# Print the number of test images and shape of the embeddings
print("Number of test images:", len(small_test_image_paths))
print("Test embeddings list shape:", small_test_embeddings.shape)

In [ ]:
# Calculate top-3 accuracy for each test image
correct_top3_count = 0
all_top3_indices = []

for i, test_embedding in enumerate(small_test_embeddings):
    closest_3_indices = top3_accuracy(test_embedding, first_embeddings)
    all_top3_indices.append(closest_3_indices)
    test_image_class = os.path.basename(os.path.dirname(small_test_image_paths[i]))
    closest_classes = [os.path.basename(os.path.dirname(first_image_paths[idx])) for idx in closest_3_indices]
    
    if test_image_class in closest_classes:
        correct_top3_count += 1
    
    # Print debug information for each test image
    print(f"Test Image {i+1}:")
    print(f"  Actual Class: {test_image_class}")
    print(f"  Predicted Top-3 Classes: {closest_classes}")
    print(f"  Correct Prediction: {test_image_class in closest_classes}")

top3_accuracy_result = correct_top3_count / len(small_test_image_paths)
print(f"\nTop-3 Accuracy: {top3_accuracy_result:.2f}")


# Test visualization with a query image
query_image_path = small_test_image_paths[0]  # Change this to any query image path from the test set
query_image_embedding = small_test_embeddings[0]

closest_3_indices = top3_accuracy(query_image_embedding, first_embeddings)
show_top3_accuracy(first_image_paths, closest_3_indices, query_image_path, mapF)